In [137]:
import pandas as pd
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction import text
import nltk
import re
from nltk import pos_tag, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/christinefang/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/christinefang/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [138]:
satp = pd.read_csv('../../../sample_events.csv')
##satp = satp[["description", "relevant_event"]]
satp = satp[["description", "internal_conflict"]]
satp = satp.drop_duplicates().reset_index(drop=True)
satp = satp.dropna()
satp.index = range(len(satp))
satp

,description,internal_conflict
0,Nagaland Post reports that a National Socialis...,1.0
1,"On February 22, Minister of State for Home, S....",0.0
2,Kanglaonline reports on June 21 that a militan...,1.0
3,Two security personnel were seriously injured ...,1.0
4,The Union Ministry of Home Affairs (UMHA) has ...,1.0
...,...,...
505,"An activist of the ruling PDP, identified as A...",1.0
506,"Earlier on March 3, a fire fight had occurred ...",1.0
507,The Kerala police recovered and later defused ...,1.0
508,A suspected terrorist lobbed a hand grenade at...,1.0


In [139]:
data_stripped = []
data_tokenized_words = []

wl = WordNetLemmatizer()

for i in range(len(satp)):
    # removes unnecessary characters
    row = re.sub('[^a-zA-Z0-9. ]', '', satp['description'][i])
    row = row.lower()
    # tokenization
    row = row.split()
    # lemmatization and stop words removal
    row = [wl.lemmatize(word) for word in row if not word in set(stopwords.words('english'))]
    row2 = re.sub('[^a-zA-Z0-9 ]', '', ' '.join(row))
    row2 = row2.split(' ')
    row3 = ' '.join(row).split('.')
    
    
    data_stripped.append(' '.join(row))
    data_tokenized_words.append(row2)


satp['stripped_description'] = data_stripped
satp['word_tokenized_description'] = data_tokenized_words

vectorizer = text.TfidfVectorizer(analyzer=lambda x: x)
X = vectorizer.fit_transform(satp["word_tokenized_description"])
print("Shape: ",X.shape)
print(X.todense())

Shape:  (510, 6256)
[[0.         0.         0.         ... 0.         0.22138964 0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [145]:
trainx = X[:400]
trainy = satp['internal_conflict'][:400]

validx = X[400:]
validy = satp["internal_conflict"][400:]
validy = validy.array

In [146]:
## Multinomial Naive Bayes
nb = MultinomialNB()
nb.fit(trainx, trainy)
prd = nb.predict(validx)

In [147]:
count = 0
for i in range(len(prd)):
    if prd[i] == validy[i]:
        count += 1
print(f"validation accuracy: {count}/{len(prd)} = {count/len(prd)}")

validation accuracy: 100/110 = 0.9090909090909091


Internal Conflict: validation accuracy = 90.91% (trained on 400 events, validation on 110)
Relevant Event: validation accuracy = 68% (trained on 550 events, )
